In [ ]:
import os
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr

In [ ]:
model_name = "llama3.2"
llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

system_message

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = llm.chat.completions.create(model = model_name, messages=messages)
    return response.choices[0].message.content


gr.ChatInterface(fn = chat, type = "messages").launch()

# Tools

In [39]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    if destination_city:
        destination_city = destination_city.lower()
    return ticket_prices.get(destination_city, "Unknown")


get_ticket_price("london")

Tool get_ticket_price called for london


'$799'

In [40]:
price_function = {
    "name": "get_ticket_price",
    "description" : "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city' ",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
        }
    
    
}
price_function

{'name': 'get_ticket_price',
 'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city' ",
 'parameters': {'type': 'object',
  'properties': {'destination_city': {'type': 'string',
    'description': 'The city that the customer wants to travel to'}},
  'required': ['destination_city'],
  'additionalProperties': False}}

In [41]:
tools = [{ "type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city' ",
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [42]:
import json
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print("tool_call", tool_call)
    arguments = json.loads(tool_call.function.arguments)
    print("arguments", arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    print("price", price)
    response = {
        "role": "tool",
        "content" : json.dumps({"destination_city": city, "price": price}),
        "tool_call_id" : tool_call.id
    }
    
    return response, city


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = llm.chat.completions.create(model = model_name, messages=messages, tools = tools)
    print("response", response)
    if  response.choices[0].finish_reason == "tools_calls":
        print("inside tool")
        message = response.choices[0].messages
        print("message", message)
        response, city = handle_tool_call(message)
        print("response", response)
        messages.append(messages)
        messages.append(response)
        print(messages)
        response = llm.chat.completions.create(model = model_name, messages=messages)
        
    return response.choices[0].message.content

gr.ChatInterface(fn = chat, type = "messages").launch()

In [ ]:
model_name = "llama3.2"
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = llm.chat.completions.create(model = model_name, messages=messages, 
                                           tools = tools
                                           )
    print("response:", response)
    if  response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = llm.chat.completions.create(model = model_name, messages=messages)
        # message = response.choices[0].message
        # response, city = handle_tool_call(message)
        # messages.append(message)
        # messages.append(response)
        # response = llm.chat.completions.create(model=model_name, messages=messages)
        
    return response.choices[0].message.content

# chat("price ticket for london", [])
gr.ChatInterface(fn = chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


response: ChatCompletion(id='chatcmpl-688', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"name":"get_ticket_price","parameters": {"destination_city":"","$ref","#e"}}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739022818, model='llama3.2', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=19, prompt_tokens=238, total_tokens=257, completion_tokens_details=None, prompt_tokens_details=None))


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = llm.chat.completions.create(model=model_name, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = llm.chat.completions.create(model=model_name, messages=messages)
    
    return response.choices[0].message.content



gr.ChatInterface(fn = chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


tool_call ChatCompletionMessageToolCall(id='call_rhhpzr0s', function=Function(arguments='{"destination_city":"any city"}', name='get_ticket_price'), type='function', index=0)
arguments {'destination_city': 'any city'}
Tool get_ticket_price called for any city
price Unknown
tool_call ChatCompletionMessageToolCall(id='call_14luirex', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function', index=0)
arguments {'destination_city': 'London'}
Tool get_ticket_price called for London
price $799
tool_call ChatCompletionMessageToolCall(id='call_0g2f8622', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function', index=0)
arguments {'destination_city': 'London'}
Tool get_ticket_price called for London
price $799
tool_call ChatCompletionMessageToolCall(id='call_zmv6ko11', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function', index=0)
arguments {'destination